In [1]:
# Set path for api_keys
import sys
sys.path.append('C:\\Users\\jtile\\Documents\\GA Tech\\Homework\\Jerry_Tilenis\\python-api-challenge')

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from ipywidgets.embed import embed_minimal_html
import gmaps
import os

import time
import json
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Import API key
from api_keys import g_key

# Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_csv_file = "../output_data/cities.csv"
cities_df = pd.read_csv(cities_csv_file)
cities_df.head()

,City ID,City,Country,City Date,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed
0,964420,port elizabeth,ZA,1593387511,-33.92,25.57,55.40,66,27,11.41
1,3440777,rocha,UY,1593387511,-34.48,-54.33,43.34,91,65,3.15
2,2094342,kavieng,PG,1593387511,-2.57,150.80,84.90,74,23,11.50
3,5879400,anchorage,US,1593387511,61.22,-149.90,59.00,54,75,6.93
4,5282297,bethel,US,1593387511,41.37,-73.41,73.00,88,40,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
cities_df = cities_df.dropna()
humidity = cities_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)
embed_minimal_html('humidity_map.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Humidity less than 25
ideal_cities_A_df = cities_df.loc[cities_df["Humidity"] < 50]

# A max temperature lower than 85 degrees but higher than 65.
ideal_cities_B_df = ideal_cities_A_df.loc[(ideal_cities_A_df["Max Temp"] < 85) & (ideal_cities_A_df["Max Temp"] > 65)]

# Clouds less than 25
ideal_cities_C_df = ideal_cities_B_df.loc[ideal_cities_B_df["Clouds"] < 25]

# Wind speed less than 15 mph.
ideal_cities_df = ideal_cities_C_df.loc[ideal_cities_C_df["Wind Speed"] < 15]

ideal_cities_df


,City ID,City,Country,City Date,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed
33,108648,qurayyat,SA,1593387514,31.33,37.34,73.40,43,0,11.41
82,2212775,sabha,LY,1593387520,27.04,14.43,82.49,22,0,10.33
104,6185377,yellowknife,CA,1593387302,62.46,-114.35,69.80,35,20,13.87
145,2123278,milkovo,RU,1593387526,54.72,158.62,70.20,46,0,3.58
155,2017155,saskylakh,RU,1593387527,71.92,114.08,83.10,34,0,5.93
167,2219235,awbari,LY,1593387529,26.59,12.78,83.35,22,0,8.19
180,5882953,aklavik,CA,1593387530,68.22,-135.01,68.00,34,20,10.29
204,3466950,carinhanha,BR,1593387533,-14.30,-43.77,73.98,47,12,6.62
233,5374376,monterey,US,1593387536,36.25,-121.25,73.99,19,0,5.01
256,140463,birjand,IR,1593387539,32.87,59.22,68.00,16,0,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Set up hotel_df
hotel_df = ideal_cities_df
#  Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


C:\Users\jtile\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Retrieving Results for Index 33: qurayyat.
Closest hotel is Mozon Furnished Aprtments.
------------
Retrieving Results for Index 82: sabha.


C:\Users\jtile\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is محطة ام الارانب -سبها مواصلات.
------------
Retrieving Results for Index 104: yellowknife.
Closest hotel is The Explorer Hotel.
------------
Retrieving Results for Index 145: milkovo.
Closest hotel is Hotel "Geologist".
------------
Retrieving Results for Index 155: saskylakh.
Missing field/result... skipping.
------------
Retrieving Results for Index 167: awbari.
Closest hotel is فندق أوباري.
------------
Retrieving Results for Index 180: aklavik.
Closest hotel is Aklavik Inn.
------------
Retrieving Results for Index 204: carinhanha.
Closest hotel is O Carinhanha Hotel.
------------
Retrieving Results for Index 233: monterey.
Missing field/result... skipping.
------------
Retrieving Results for Index 256: birjand.
Closest hotel is Birjand Tourist Hotel.
------------
Retrieving Results for Index 295: tiznit.
Missing field/result... skipping.
------------
Retrieving Results for Index 304: yumen.
Missing field/result... skipping.
------------
Retrieving Results for Inde

In [7]:
# Get rid of records without hotels
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df

locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
embed_minimal_html('hotel_map.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))